In [ ]:
import numpy as np
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('../custom_components/solar_battery_forecast/brains/'))
if module_path not in sys.path:
    sys.path.append(module_path)

from statsmodels.tsa.api import STLForecast
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.datasets import macrodata
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing

from load_forecaster import LoadForecaster
pd.set_option('display.max_rows', None)

df = pd.read_csv('entities-2023-09-06_10 17 37.csv', usecols=['start', 'mean'])
# df = pd.read_csv('entities-2023-09-02_17 33 41.csv', usecols=['start', 'mean'])
df['start'] = pd.to_datetime(df['start'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index('start')
df = df.asfreq(freq='H')
df = df[['mean']].tail(9*7*24)

train_period = 24 * 7 * 6
for x in range(0, len(df) - train_period, 24):
    train_start = x
    train_end = x + train_period
    print(f"Training from {train_start} - {train_end - 1}: {df.iloc[train_start].name} - {df.iloc[train_end - 1].name}")
    f = LoadForecaster()
    prediction = f.predict(df.iloc[train_start:train_end].copy())
    df = df.combine_first(prediction)


In [ ]:
pyplot.figure()
fig, ax = pyplot.subplots(figsize=(15, 5))
df[train_period-48:]['mean'].plot(ax=ax)
df[train_period-48:]['predicted'].plot(ax=ax)
ax.fill_between(df.index, df['predicted_lower'], df['predicted_upper'], color='k', alpha=0.1);  
pyplot.legend()
pyplot.show()

daily_sums = df[train_period:].resample('D').sum()
print(daily_sums)
print(mean_squared_error(daily_sums['mean'], daily_sums['predicted']))
print(mean_absolute_error(daily_sums['mean'], daily_sums['predicted']))